In [ ]:
from pyquery import PyQuery as pq
def getDBLP(URL):
    doc=pq(url=URL)
    det= doc(".visit.drop-down")
    data={'site':det('.head a').attr('href')}
    
    links=det('.body a')
    for l in links:
        text=pq(l).text()
        if(text):
            data[text]=l.attrib['href']
    if len(links)<3:
        all=[doc('#index-side-panel').next().find('a'),
        doc('#info-section').next().find('a')]
        
        for item in all:
            if(item):
                data[item.text()]=item.attr('href')
        
        
    if not(data['site']): 
        for p in data:
            if(data[p]):
                data['site']=data[p]
                break
    return data
import json
all=json.load(open('core-jnls.json'))

# getDBLP('https://dblp.uni-trier.de/db/conf/ant/index.html')

In [ ]:
i=0
import time
for item in all:
    i+=1
    if(item['DBLP Source'] and not('Links' in item)):
#         print(item['Title'])
        try:
            item['Links']=getDBLP(item['DBLP Source'])
            print(f"{i}\t{item['Title']}==== {item['Links']['site']}")
        except:
            print(f'error {item["Title"]}')
        time.sleep(1)
#         break


In [ ]:
with open('core-jnls.json', 'w') as fp:
    json.dump(all, fp)

In [ ]:
#!pip install pyquery
from pyquery import PyQuery as pq
import json

def getCoreData(conf, id):
    if(conf):
        TYPE = 'Conference'
        URL = f'http://portal.core.edu.au/conf-ranks/{id}'
    else:
        TYPE = 'Journal'
        URL = f'http://portal.core.edu.au/jnl-ranks/{id}'
    doc=pq(url=URL)
    det= doc("#detail")
    
    
    data={
    'CoreId':id,
    'Type':TYPE,
    'Title':det('h2:first').text(),
    'Acronym':'',
    'Acronym2':'',
    'DBLP Source':'',
    'Rank':'',
    'Rank Source':'',
    'Field':[],
    'Info':[]    
    }

    details=det('.detail')
    for i in range(len(details)):
        rows=details.eq(i)('.row')
        row=rows.eq(0)
        splt=row.text().split(':')
        titr=splt[0].strip()
        val=row.text()[len(splt[0])+1:].strip()

        if titr=='Acronym':
            data[titr]=val
        elif titr=='DBLP Source':
            if(val!="N/A"):
                data[titr]=val
                data['Acronym2']=val.split('/')[-1]
        elif titr=='Source':
            info={'Source':'','Rank':'','Field Of Research':[]}
            for j in range(len(rows)):
                row=rows.eq(j)
                splt=row.text().split(':')
                titr=splt[0].strip()
                val=row.text()[len(splt[0])+1:].strip()

                if titr=='Field Of Research':
                    val=val.replace('†','')
                    info['Field Of Research'].append(val)                
                    if not (val in data['Field']): data['Field'].append(val)
                elif val=='':
                    info['other']=row.html().strip().replace('\n','').replace('  ','').replace('a href="/core','a href="http://portal.core.edu.au/core')
                else:
                    if(titr=='Rank' and data['Rank']==''): 
                        data['Rank']=val
                        data['Rank Source']=info['Source']
                    info[titr]=val
            data['Info'].append(info)

        
        if not(conf):
            doc=pq(url=f'http://portal.core.edu.au/jnl-ranks/?search={data["Title"]}&by=all&source=all&sort=atitle&page=1')
            det= doc("table tr td a").attr('href')
            if not (det == None):
                data['DBLP Source']=det
                data['Acronym2']=det.split('/')[-1]
            
           
    return data

confs=[]
for i in range(2302):
    print(i,end=' ')
    try:
        confs.append(getCoreData(conf=True,id=i))
    except Exception as e:
        print(e)

with open('core-conf2021.json', 'w') as fp:
    json.dump(confs, fp)

# jnls=[]
# for i in range(937):
#     print(i,end=' ')
#     try:
#         jnls.append(getCoreData(conf=False,id=i))
#     except Exception as e:
#         print(e)
    

# with open('core-jnls2021.json', 'w') as fp:
#     json.dump(jnls, fp)    
    
# import csv
# with open('core2020.tsv', 'wt') as out_file:
#     tsv_writer = csv.writer(out_file, delimiter='\t')
#     tsv_writer.writerow(['name', 'field'])
    
    


In [ ]:
import pandas as pd
def get_scimagojr(year=2019):
    info=json.load(open('scimagojr_info.json'))
    print(info['area']['1000'])
    url=f'https://www.scimagojr.com/shapeofscience/ajaxmap.php?authorcountry=0&year={year}'
    df=pd.read_csv(url,delimiter=';')
    data={}
    for k,row in df.iterrows():
#         cats=row['CATEGORIAS'].split(',')
#         areas=row['AREAS'].split(',')
        
#         obj={
#             'Id':row['SOURCEID'],
#             'Title':row['SOURCETITLE'],
            
#             'SJR':row['SJR'],
            
#         }
        forbiden=['posX','posY','Cluster','wDegree','CATEGORIAS','AREAS','SCIELO']
        obj={k:row[k] for k in row.keys() if not(k in forbiden)}
        obj['Topic']={y:[x for x in row['CATEGORIAS'].split(',') if x[0:2]==y[0:2]] for y in row['AREAS'].split(',')}
        
        data[row['SOURCEID']]=obj
        
    with open(f'scimagojr_{year}.json', 'w') as fp:
        json.dump(data, fp)
    return pd.DataFrame.from_dict(data, orient='index')
a=get_scimagojr(2019)
a

In [ ]:
import pandas as pd
import numpy as np
def get_scimagojr_journal(year=2020):
    info=json.load(open('scimagojr_info.json'))
    
    url=f'https://www.scimagojr.com/journalrank.php?year={year}&type=j&area=1700&out=xls'
    df=pd.read_csv(url,delimiter=';')
    data=[]
    for k,row in df.iterrows():
#         cats=row['CATEGORIAS'].split(',')
#         areas=row['AREAS'].split(',')
        
#         obj={
#             'Id':row['SOURCEID'],
#             'Title':row['SOURCETITLE'],
            
#             'SJR':row['SJR'],
            
#         }
        forbiden=['posX','posY','Cluster','wDegree','CATEGORIAS','AREAS','SCIELO']
        obj={k:row[k] if row[k]==row[k] else '' for k in row.keys() if not(k in forbiden)}
#         obj['Topic']={y:[x for x in row['CATEGORIAS'].split(',') if x[0:2]==y[0:2]] for y in row['AREAS'].split(',')}
#         print(obj)
        data.append(obj)
        
    with open(f'scimagojr_journal_{year}.json', 'w') as fp:
        json.dump(data, fp)
    return data;
a=get_scimagojr_journal(2020)
a


In [ ]:
import numpy as np
np.nan=='a'

In [ ]:
import pandas as pd
import numpy as np
def update_citescore_journal():
    info=json.load(open('scimagojr_journal_2020.json'))
    for d in info:
        d['CiteScore']=0
        d['SNIF']=0
#         d['Percentile']=0
    idm={d['Sourceid']:d for d in info}
    
    url=f'citescore.csv'
    df=pd.read_csv(url)
    data=[]
    for k,row in df.iterrows():
#         cats=row['CATEGORIAS'].split(',')
#         areas=row['AREAS'].split(',')
        id=row['Scopus Source ID']
        
        if(id in idm):
#             print(row)
            d=idm[id]
            d['CiteScore']=row['CiteScore 2020']
            d['SNIP']=row['SNIP']
#             d['Percentile']=row['Percentile']
        else: 
#             print(row)
            pass
            
#         obj={
#             'Id':row['SOURCEID'],
#             'Title':row['SOURCETITLE'],
            
#             'SJR':row['SJR'],
            
#         }
        
    with open(f'scimagojr_journal_2020_cs.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_citescore_journal()
a


In [ ]:
import pandas as pd
import numpy as np
from pyquery import PyQuery as pq

def update_link_journal():
    info=json.load(open('scimagojr_journal_2020_cs_link.json'))
    try:
#         for d in info:
#             d['Scope']=''
        for d in info:
            if(d.get('Scope','')!=''):continue
            print(f"{d['Sourceid']}:{d['Title']}")
    #         d['Site']=
            
            d['Links']={}
            d['Scope']=''
            doc=pq(url=f'https://www.scimagojr.com/journalsearch.php?q={d["Sourceid"]}&tip=sid&clean=0')
            links=doc('#question_journal')
            for l in links:
                d['Links'][l.text]=l.attrib['href']

            x=doc('div.fullwidth')[0]
            txt=pq(x).text()
            for c in x.getchildren():
                txt=txt.replace(pq(c).text(),'')
            d['Scope']=txt.strip()
    #         d['Site']=d['Links'].get('Homepage','')
    #         if not d['Site']:
    #             for p in d['Links']:
    #                 d['Site']=d['Links'][p]
    #                 break

    #         print(d)    
    #         return
    except Exception as e:
        print(e)
    with open(f'scimagojr_journal_2020_cs_link.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_link_journal()



In [ ]:
int(1.1)

In [ ]:
import pandas as pd
import numpy as np
from pyquery import PyQuery as pq

def update_reviw_time():
    info=json.load(open('scimagojr_journal_2020_cs_link.json'))
    
    for d in info:
#             if(d.get('ReviewFirst','')!=''):
#                  del d['ReviewFirst']
#                  del d['ReviewFinal']
#             continue
            if ('elsevier' in d['Links'].get('Homepage','')):
                print(f"{d['Sourceid']}:{d['Title']}")# {d['Links'].get('Homepage','')}")
                for Issn in d['Issn'].split(', '):
                    issn2=(f'{Issn[0:4]}-{Issn[4:8]}')
                    try:
                        d['Review']={}
                        doc=pq(f'https://journalinsights.elsevier.com/journals/{issn2}/review_speed')
                        for i in range(1,4):
                            td=doc(f'section.graph-data.linegraph tbody tr:nth-child({i}) td')
                            d['Review'][pq(td[0]).text()]={
                                'First':int(float(pq(td[1]).text())*7),
                                'Final':int(float(pq(td[2]).text())*7)
                            }

                    except  Exception as e:
                        print(e)
                    try:
                        d['Acceptance']={}
                        doc=pq(f'https://journalinsights.elsevier.com/journals/{issn2}/acceptance_rate')
                        for i in range(1,4):
                            td=doc(f'section.graph-data.percentage_bars tbody tr:nth-child({i}) td')
                            d['Acceptance'][pq(td[0]).text()]={
                                'Submission':int(float(pq(td[1]).text())),
                                'Accept':int(float(pq(td[2]).text())),
                                'Rate':int(float(pq(td[3]).text()))
                            }
                        print(d)

                    except  Exception as e:
                        print(e)
                        
            if ('springer2343243242' in d['Links'].get('Homepage','')):
                try:
                        d['Review']={}
                        doc=pq(d['Links']['Homepage'])
                        sel=lambda p:doc(f"dd.app-journal-metrics__details[data-test='"+p+"']").text()
                        td=doc(f"dd.app-journal-metrics__details[data-test='impact-factor-value']")
#                         year=sel('impact-factor-value').split(' ')[1][1:5]
                        year=2020
                        IF=sel('impact-factor-value').split(' ')[0]
                        IF5=sel('five-year-impact-factor-value').split(' ')[0]
                        first=sel('metrics-speed-value').split(' ')[0]
                        final=sel('metrics-acceptance-time-value').split(' ')[0]
                        
                        if(first or final):
                            d['Review'][year]={}
                        if first: d['Review'][year]['First']=int(first)
                        if final: d['Review'][year]['Final']=int(final)
                            
                        print(f"{d['Sourceid']}:{d['Title']}")# {d['Links'].get('Homepage','')}")
                except  Exception as e:
                        print(e)
                        print(d['Links'].get('Homepage',''))
                        
    
    with open(f'scimagojr_journal_2020_cs_link.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_reviw_time()



In [ ]:
import pandas as pd
import numpy as np

def update_aut_rank():
    info=json.load(open('scimagojr_journal_2020_cs_link.json'))
    aut_q=pd.read_csv('aut-q-2020.csv')
    aut_q['Qnum']=5-aut_q['Q'].str.replace('Q','').astype(int)
    aut_q_m=aut_q.groupby('ISSN').mean()[['Qnum']]
    aut_q_m['Qnum']='Q'+(5-aut_q_m['Qnum'].round().astype(int)).astype(str)
    aut_q_m.index=aut_q_m.index.str.replace('-','')
    
    for d in info:
       if d['Issn']:
         for issn in d['Issn'].split(', '):
            if issn in aut_q_m.index:
                d['aut']=aut_q_m.loc[issn]['Qnum']
    
    with open(f'scimagojr_journal_2020_cs_link_aut.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_aut_rank()



In [ ]:
a

In [ ]:
doc=pq(f'https://www.springer.com/journal/466')


In [ ]:
issn2='0168-874X'
d={}
d['Acceptance']={}
doc=pq(f'https://journalinsights.elsevier.com/journals/{issn2}/acceptance_rate')
for i in range(1,4):
    td=doc(f'section.graph-data.percentage_bars tbody tr:nth-child({i}) td')
    print(td)
    d['Acceptance'][pq(td[0]).text()]={
        'Submission':int(float(pq(td[1]).text())),
        'Accept':int(float(pq(td[2]).text())),
        'Rate':int(float(pq(td[3]).text()))
    }
    
d
